# Costruzione dataset Blazars per Neural Network
### Indagine sul catalogo OU4Blazar Master List e 3HSP

### Per utilizzare il notebook scaricare i cataloghi v1 e v2 dal OU4Blazar Master list e il catalogo 3HSP in csv eliminando virgolette e spazi non necessari.

In [1]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
print(sns.__version__)

import pandas as pd
print(pd.__version__)

0.11.2
1.3.5


In [2]:
%ls

3hsp.csv                                new_veron.csv
milliquas_radio_galaxies_catalogue.txt  ou4bl3hsp_name.csv
Milliquas-ReadMe.txt                    ou4bl3hsp_radec.csv
milliquas.txt                           OU4Blazar.ipynb
my3hsp_name_not_recognized.csv          OU4Bl_v1.csv
my3hsp_name_prova.csv                   OU4Bl_v2.csv
new_milliquas.csv                       veron_catalogue.txt


In [3]:
ou4bl = pd.read_csv('OU4Bl_v2.csv', encoding='utf-8-sig')
ou4bl_old= pd.read_csv('OU4Bl_v1.csv', encoding='utf-8-sig')

In [4]:
ou4bl

,id,Source name,R.A.,Dec,Radio flux d.,Fermi flux,Log nu_peak,Redshift,Original catalog
0,1,3HSPJ000236.0-081532.4,00 02 36.06,-08 15 32.4,29.0,0.000000e+00,16.0,0.390,3HSP
1,2,5BZBJ0002-0024,00 02 57.16,-00 24 47.01,178.0,0.000000e+00,0.0,0.523,5BZCAT
2,3,4FGLJ0003.3-5905,00 03 13.32,-59 05 47.61,505.9,1.680000e-12,0.0,0.000,Fermi 4LAC
3,4,4FGLJ0003.3-1928,00 03 18.66,-19 27 22.28,171.0,2.590000e-12,0.0,0.000,Fermi 4LAC
4,5,5BZQJ0003+2129,00 03 19.32,+21 29 44.48,83.0,0.000000e+00,0.0,0.450,5BZCAT
...,...,...,...,...,...,...,...,...,...
6072,6073,4FGLJ0001.5+2113,00 01 32.4,+21 13 36.4,217.9,1.930000e-11,0.0,0.000,Fermi 3LAC
6073,6074,3HSPJ000132.7-415525.1,00 01 32.74,-41 55 25.1,13.2,3.390000e-12,15.8,0.000,3HSP
6074,6075,5BZGJ0001-1031,00 01 57.24,-10 31 17.18,41.0,0.000000e+00,0.0,0.252,5BZCAT
6075,6076,3HSPJ000158.1-115047.0,00 01 58.14,-11 50 47.0,3.3,0.000000e+00,16.2,0.530,3HSP


In [5]:
# ou4bl.sample(10)
ou4bl_old

,id,Source name,R.A.,Dec,Radio flux d.,Fermi flux,Log nu_peak,Redshift,Original catalog
0,1,3HSPJ000236.0-081532.4,00 02 36.06,-08 15 32.4,29.0,0.000000e+00,16.0,0.390,3HSP
1,2,5BZBJ0002-0024,00 02 57.16,-00 24 47.01,178.0,0.000000e+00,0.0,0.523,BZCAT 5
2,3,5BZQJ0003+2129,00 03 19.32,+21 29 44.48,83.0,0.000000e+00,0.0,0.450,BZCAT 5
3,4,3HSPJ000319.5-524727.4,00 03 19.56,-52 47 27.38,65.3,2.870000e-10,16.9,0.370,3HSP
4,5,3FGLJ0003.8-1151,00 03 42.83,-11 51 49.28,352.5,2.770000e-10,0.0,0.000,Fermi 3LAC
...,...,...,...,...,...,...,...,...,...
5335,5336,3FGLJ0001.4+2120,00 01 32.4,+21 13 36.4,217.9,9.810000e-10,0.0,0.000,Fermi 3LAC
5336,5337,3HSPJ000132.7-415525.1,00 01 32.74,-41 55 25.1,13.2,2.630000e-10,15.8,0.000,3HSP
5337,5338,5BZGJ0001-1031,00 01 57.24,-10 31 17.18,41.0,0.000000e+00,0.0,0.252,BZCAT 5
5338,5339,3HSPJ000158.1-115047.0,00 01 58.14,-11 50 47.0,3.3,0.000000e+00,16.2,0.530,3HSP


In [6]:
# converti coordinate
def sex2deg(ra,dec):
    h,m,s = list(map(float,ra.split()))
    ra = s / 86400
    ra += m / 1440
    ra += h / 24
    ra *= 360
    
    d,m,s = dec.split()
    dec = float(s) / 3600
    dec += float(m) / 60
    if d[0] == '-':
        dec = float(d) - dec
    else:
        dec = float(d) + dec
    
    return ra,dec

In [7]:
RA_COL = 'R.A.'
DEC_COL = 'Dec'
def apply_df(df):
    try:
        bla = list(sex2deg(df[RA_COL],df[DEC_COL]))
    except:
        bla = [None,None]
    return bla

ou4bl[['RA','DEC']] = ou4bl.apply(apply_df, axis=1, result_type = 'expand')
ou4bl_old[['RA','DEC']] = ou4bl_old.apply(apply_df, axis=1, result_type = 'expand')

In [8]:
ou4bl

,id,Source name,R.A.,Dec,Radio flux d.,Fermi flux,Log nu_peak,Redshift,Original catalog,RA,DEC
0,1,3HSPJ000236.0-081532.4,00 02 36.06,-08 15 32.4,29.0,0.000000e+00,16.0,0.390,3HSP,0.650250,-8.259000
1,2,5BZBJ0002-0024,00 02 57.16,-00 24 47.01,178.0,0.000000e+00,0.0,0.523,5BZCAT,0.738167,-0.413058
2,3,4FGLJ0003.3-5905,00 03 13.32,-59 05 47.61,505.9,1.680000e-12,0.0,0.000,Fermi 4LAC,0.805500,-59.096558
3,4,4FGLJ0003.3-1928,00 03 18.66,-19 27 22.28,171.0,2.590000e-12,0.0,0.000,Fermi 4LAC,0.827750,-19.456189
4,5,5BZQJ0003+2129,00 03 19.32,+21 29 44.48,83.0,0.000000e+00,0.0,0.450,5BZCAT,0.830500,21.495689
...,...,...,...,...,...,...,...,...,...,...,...
6072,6073,4FGLJ0001.5+2113,00 01 32.4,+21 13 36.4,217.9,1.930000e-11,0.0,0.000,Fermi 3LAC,0.385000,21.226778
6073,6074,3HSPJ000132.7-415525.1,00 01 32.74,-41 55 25.1,13.2,3.390000e-12,15.8,0.000,3HSP,0.386417,-41.923639
6074,6075,5BZGJ0001-1031,00 01 57.24,-10 31 17.18,41.0,0.000000e+00,0.0,0.252,5BZCAT,0.488500,-10.521439
6075,6076,3HSPJ000158.1-115047.0,00 01 58.14,-11 50 47.0,3.3,0.000000e+00,16.2,0.530,3HSP,0.492250,-11.846389


In [9]:
ou4bl_old

,id,Source name,R.A.,Dec,Radio flux d.,Fermi flux,Log nu_peak,Redshift,Original catalog,RA,DEC
0,1,3HSPJ000236.0-081532.4,00 02 36.06,-08 15 32.4,29.0,0.000000e+00,16.0,0.390,3HSP,0.650250,-8.259000
1,2,5BZBJ0002-0024,00 02 57.16,-00 24 47.01,178.0,0.000000e+00,0.0,0.523,BZCAT 5,0.738167,-0.413058
2,3,5BZQJ0003+2129,00 03 19.32,+21 29 44.48,83.0,0.000000e+00,0.0,0.450,BZCAT 5,0.830500,21.495689
3,4,3HSPJ000319.5-524727.4,00 03 19.56,-52 47 27.38,65.3,2.870000e-10,16.9,0.370,3HSP,0.831500,-52.790939
4,5,3FGLJ0003.8-1151,00 03 42.83,-11 51 49.28,352.5,2.770000e-10,0.0,0.000,Fermi 3LAC,0.928458,-11.863689
...,...,...,...,...,...,...,...,...,...,...,...
5335,5336,3FGLJ0001.4+2120,00 01 32.4,+21 13 36.4,217.9,9.810000e-10,0.0,0.000,Fermi 3LAC,0.385000,21.226778
5336,5337,3HSPJ000132.7-415525.1,00 01 32.74,-41 55 25.1,13.2,2.630000e-10,15.8,0.000,3HSP,0.386417,-41.923639
5337,5338,5BZGJ0001-1031,00 01 57.24,-10 31 17.18,41.0,0.000000e+00,0.0,0.252,BZCAT 5,0.488500,-10.521439
5338,5339,3HSPJ000158.1-115047.0,00 01 58.14,-11 50 47.0,3.3,0.000000e+00,16.2,0.530,3HSP,0.492250,-11.846389


In [10]:
#ou4bl.dtypes

#type(ou4bl.index)

# non necessario con il catalogo ou4 ripulito da " e spazi non necessari
# farlo se groupby non funziona
# ou_bl.columns = ou_bl.columns.map(str.strip)
# ou_bl.columns

In [11]:
ou4bl.columns

Index(['id', 'Source name', 'R.A.', 'Dec', 'Radio flux d.', 'Fermi flux',
       'Log nu_peak', 'Redshift', 'Original catalog', 'RA', 'DEC'],
      dtype='object')

In [12]:
ou4bl.groupby('Original catalog').size().sort_values(ascending=False)

Original catalog
5BZCAT        3561
3HSP          1353
Fermi 4LAC     748
Fermi 3LAC     415
dtype: int64

In [13]:
# ou4bl.tail()
ou4bl_old.groupby('Original catalog').size().sort_values(ascending=False)

Original catalog
BZCAT 5       3561
3HSP          1353
Fermi 3LAC     415
 ?              11
dtype: int64

In [14]:
# define the colums of interest
col = dict(idn='id',
           ra='RA',
           dec='DEC',
           name='Source name',
           redshift='Redshift',
           flux_radio='Radio flux d.',
           fermi_flux='Fermi flux',
           nupk='Log nu_peak',
           catalog='Original catalog'
          )

In [16]:
# seleziono catalogo 3HSP e tengo solo le colonne di interesse
ou4bl_3HSP = ou4bl.loc[ou4bl[col['catalog']] == '3HSP']
ou4bl_3HSP = ou4bl_3HSP[list(col.values())]
# salvo file solo 3HSP per il run con il sed builder
ou4bl_3HSP[col['name']].to_csv('ou4bl3hsp_name.csv', encoding='utf-8-sig', index=False)
ou4bl_3HSP

,id,RA,DEC,Source name,Redshift,Radio flux d.,Fermi flux,Log nu_peak,Original catalog
0,1,0.650250,-8.259000,3HSPJ000236.0-081532.4,0.390,29.0,0.000000e+00,16.0,3HSP
5,6,0.831500,-52.790939,3HSPJ000319.5-524727.4,0.370,65.3,3.600000e-12,16.9,3HSP
13,14,1.307167,-26.243919,3HSPJ000513.7-261438.2,0.320,37.4,0.000000e+00,15.4,3HSP
16,17,1.470750,-28.751058,3HSPJ000552.9-284503.9,0.570,5.0,0.000000e+00,15.4,3HSP
21,22,1.612000,1.602919,3HSPJ000626.9+013610.6,0.787,7.5,1.970000e-12,16.4,3HSP
...,...,...,...,...,...,...,...,...,...
6067,6068,0.318167,29.592939,3HSPJ000116.3+293534.7,0.580,4.0,0.000000e+00,16.0,3HSP
6068,6069,0.321083,-31.845500,3HSPJ000117.0-315043.8,0.450,7.2,0.000000e+00,15.5,3HSP
6073,6074,0.386417,-41.923639,3HSPJ000132.7-415525.1,0.000,13.2,3.390000e-12,15.8,3HSP
6075,6076,0.492250,-11.846389,3HSPJ000158.1-115047.0,0.530,3.3,0.000000e+00,16.2,3HSP


In [17]:
ou4bl_3HSP[[col['ra'],col['dec']]].to_csv('ou4bl3hsp_radec.csv', encoding='utf-8-sig', index=False)

In [18]:
ou4bl_5BZCAT = ou4bl.loc[ou4bl['Original catalog'] == '5BZCAT']
ou4bl_5BZCAT = ou4bl_5BZCAT[list(col.values())]
ou4bl_5BZCAT

,id,RA,DEC,Source name,Redshift,Radio flux d.,Fermi flux,Log nu_peak,Original catalog
1,2,0.738167,-0.413058,5BZBJ0002-0024,0.523,178.0,0.000000e+00,0.0,5BZCAT
4,5,0.830500,21.495689,5BZQJ0003+2129,0.450,83.0,0.000000e+00,0.0,5BZCAT
9,10,1.020250,-11.816108,5BZBJ0004-1148,0.000,459.0,3.200000e-12,0.0,5BZCAT
10,11,1.067292,46.255028,5BZQJ0004+4615,1.810,120.0,0.000000e+00,0.0,5BZCAT
11,12,1.148958,20.328497,5BZQJ0004+2019,0.677,55.0,0.000000e+00,0.0,5BZCAT
...,...,...,...,...,...,...,...,...,...
6065,6066,0.272042,-15.851919,5BZQJ0001-1551,2.044,348.0,0.000000e+00,0.0,5BZCAT
6066,6067,0.285917,19.242808,5BZQJ0001+1914,3.100,265.0,0.000000e+00,0.0,5BZCAT
6069,6070,0.325000,-7.774058,5BZBJ0001-0746,0.000,208.0,9.440000e-12,0.0,5BZCAT
6071,6072,0.339500,-0.194419,5BZBJ0001-0011,0.462,111.0,0.000000e+00,0.0,5BZCAT


In [19]:
# concateno i due df per avere il mio elenco completo di sorgenti blazars
mybl_list = pd.concat([ou4bl_3HSP, ou4bl_5BZCAT], ignore_index=True)

mybl_list

,id,RA,DEC,Source name,Redshift,Radio flux d.,Fermi flux,Log nu_peak,Original catalog
0,1,0.650250,-8.259000,3HSPJ000236.0-081532.4,0.390,29.0,0.000000e+00,16.0,3HSP
1,6,0.831500,-52.790939,3HSPJ000319.5-524727.4,0.370,65.3,3.600000e-12,16.9,3HSP
2,14,1.307167,-26.243919,3HSPJ000513.7-261438.2,0.320,37.4,0.000000e+00,15.4,3HSP
3,17,1.470750,-28.751058,3HSPJ000552.9-284503.9,0.570,5.0,0.000000e+00,15.4,3HSP
4,22,1.612000,1.602919,3HSPJ000626.9+013610.6,0.787,7.5,1.970000e-12,16.4,3HSP
...,...,...,...,...,...,...,...,...,...
4909,6066,0.272042,-15.851919,5BZQJ0001-1551,2.044,348.0,0.000000e+00,0.0,5BZCAT
4910,6067,0.285917,19.242808,5BZQJ0001+1914,3.100,265.0,0.000000e+00,0.0,5BZCAT
4911,6070,0.325000,-7.774058,5BZBJ0001-0746,0.000,208.0,9.440000e-12,0.0,5BZCAT
4912,6072,0.339500,-0.194419,5BZBJ0001-0011,0.462,111.0,0.000000e+00,0.0,5BZCAT


In [20]:
mybl_list.to_csv("mybl_list.csv", encoding='utf-8-sig', index=False)

In [21]:
# composizione del SET BLAZARS per rete neurale
mybl_list.groupby(col['catalog']).size().sort_values(ascending=False)

Original catalog
5BZCAT    3561
3HSP      1353
dtype: int64

### Problemi con riconoscimento nomi 3HSP presenti nel catalogo OU4 sul SED Builder
Scarico direttamente il catalogo 3HSP e lo epuro dalle sorgenti già presenti nel 5BZcat

In [22]:
cat3hsp=pd.read_csv('3hsp.csv',encoding='utf-8-sig')

In [23]:
cat3hsp

,id,3HSP Source name,BZcat name,R.A.,Dec,NVSS/FIRSTflux density(mJy),X-ray flux d. @ 1keV (μJy),Log(peak flux),γ-ray counterpart,Fermi γ-ray slope,2WHSP name
0,1,3HSPJ000116.4+293534,NaN,0.31821,29.59278,4.0,NaN,-12.9?,NaN,NaN,2WHSPJ000116.3+293534
1,2,3HSPJ000117.1-315043,NaN,0.32117,-31.84528,7.2,0.23,-12.3?,NaN,NaN,2WHSPJ000116.9-315043
2,3,3HSPJ000132.7-415525,NaN,0.38642,-41.92361,13.2,0.34,-11.7,4FGL J0001.6-4156,2.09,2WHSPJ000132.6-415524
3,4,3HSPJ000158.1-115047,NaN,0.49225,-11.84639,3.3,0.11,-12.5,NaN,NaN,2WHSPJ000158.0-115047
4,5,3HSPJ000215.2-672653,NaN,0.56329,-67.44806,21.4,0.27,-11.8,4FGL J0002.1-6728,1.95,2WHSPJ000215.1-672653
...,...,...,...,...,...,...,...,...,...,...,...
2008,2009,3HSPJ235907.9-303740,5BZBJ2359-3037,359.78283,-30.62778,66.7,12.64,-11,4FGL J2359.0-3038,2.02,2WHSPJ235907.8-303740
2009,2010,3HSPJ235917.0+021520,NaN,359.82100,2.25556,5.0,0.10,-12.7,NaN,NaN,2WHSPJ235916.9+021520
2010,2011,3HSPJ235919.5-204756,NaN,359.83138,-20.79889,472.7,0.24,-12.2,4FGL J2359.3-2049,2.02,NaN
2011,2012,3HSPJ235921.3-131129,NaN,359.83892,-13.19139,30.4,0.07,-12.7,NaN,NaN,2WHSPJ235921.3-131129


In [24]:
# controlla una specifica riga del data frame
#cat3hsp.iloc[2007]

In [27]:
# seleziona righe del dataframe non presenti nel catalogo 5BZcat
my_3hsp = cat3hsp.loc[cat3hsp['BZcat name'].isnull()]

In [28]:
my_3hsp

,id,3HSP Source name,BZcat name,R.A.,Dec,NVSS/FIRSTflux density(mJy),X-ray flux d. @ 1keV (μJy),Log(peak flux),γ-ray counterpart,Fermi γ-ray slope,2WHSP name
0,1,3HSPJ000116.4+293534,NaN,0.31821,29.59278,4.0,NaN,-12.9?,NaN,NaN,2WHSPJ000116.3+293534
1,2,3HSPJ000117.1-315043,NaN,0.32117,-31.84528,7.2,0.23,-12.3?,NaN,NaN,2WHSPJ000116.9-315043
2,3,3HSPJ000132.7-415525,NaN,0.38642,-41.92361,13.2,0.34,-11.7,4FGL J0001.6-4156,2.09,2WHSPJ000132.6-415524
3,4,3HSPJ000158.1-115047,NaN,0.49225,-11.84639,3.3,0.11,-12.5,NaN,NaN,2WHSPJ000158.0-115047
4,5,3HSPJ000215.2-672653,NaN,0.56329,-67.44806,21.4,0.27,-11.8,4FGL J0002.1-6728,1.95,2WHSPJ000215.1-672653
...,...,...,...,...,...,...,...,...,...,...,...
2007,2008,3HSPJ235836.8-180717,NaN,359.65346,-18.12139,10.2,0.13,-11.8,4FGL J2358.5-1808,1.82,NaN
2009,2010,3HSPJ235917.0+021520,NaN,359.82100,2.25556,5.0,0.10,-12.7,NaN,NaN,2WHSPJ235916.9+021520
2010,2011,3HSPJ235919.5-204756,NaN,359.83138,-20.79889,472.7,0.24,-12.2,4FGL J2359.3-2049,2.02,NaN
2011,2012,3HSPJ235921.3-131129,NaN,359.83892,-13.19139,30.4,0.07,-12.7,NaN,NaN,2WHSPJ235921.3-131129


In [29]:
#strange_3hsp = my_3hsp.loc[my_3hsp['3HSP Source name'].str.startswith('?')].copy()
#strange_3hsp

In [30]:
name_my3hsp=my_3hsp['3HSP Source name']

In [31]:
name_my3hsp.to_csv('my3hsp_name.csv', encoding='utf-8-sig', index=False, header=None)

In [32]:
# non tutte le sorgenti 3HSP vengono riconosciute quando inserite nel SED Builder
# passo al catalogo 2WHSP, togliendo le righe "NaN"
my_2whsp = my_3hsp.loc[my_3hsp['2WHSP name'].notnull()]
my_2whsp

,id,3HSP Source name,BZcat name,R.A.,Dec,NVSS/FIRSTflux density(mJy),X-ray flux d. @ 1keV (μJy),Log(peak flux),γ-ray counterpart,Fermi γ-ray slope,2WHSP name
0,1,3HSPJ000116.4+293534,NaN,0.31821,29.59278,4.0,NaN,-12.9?,NaN,NaN,2WHSPJ000116.3+293534
1,2,3HSPJ000117.1-315043,NaN,0.32117,-31.84528,7.2,0.23,-12.3?,NaN,NaN,2WHSPJ000116.9-315043
2,3,3HSPJ000132.7-415525,NaN,0.38642,-41.92361,13.2,0.34,-11.7,4FGL J0001.6-4156,2.09,2WHSPJ000132.6-415524
3,4,3HSPJ000158.1-115047,NaN,0.49225,-11.84639,3.3,0.11,-12.5,NaN,NaN,2WHSPJ000158.0-115047
4,5,3HSPJ000215.2-672653,NaN,0.56329,-67.44806,21.4,0.27,-11.8,4FGL J0002.1-6728,1.95,2WHSPJ000215.1-672653
...,...,...,...,...,...,...,...,...,...,...,...
2000,2001,3HSPJ235116.1-760015,NaN,357.81721,-76.00417,68.1,NaN,-12.2,4FGL J2351.3-7559,1.69,2WHSPJ235116.0-760015
2004,2005,3HSPJ235725.1-171234,NaN,359.35471,-17.20944,7.9,0.03,-12.6,NaN,1.80,2WHSPJ235725.0-171233
2009,2010,3HSPJ235917.0+021520,NaN,359.82100,2.25556,5.0,0.10,-12.7,NaN,NaN,2WHSPJ235916.9+021520
2011,2012,3HSPJ235921.3-131129,NaN,359.83892,-13.19139,30.4,0.07,-12.7,NaN,NaN,2WHSPJ235921.3-131129


### I 3HSP non BZcat erano 1356. Selezionando le sorgenti con nome 2WHSP scendiamo a 1072 sorgenti:
### perdiamo 284 blazars

In [34]:
whsp_name=my_2whsp['2WHSP name']
whsp_name.to_csv('my2whsp_name.csv', encoding='utf-8-sig', index=False, header=None)